In [1]:
import sqlite3
import pandas as pd
from pathlib import Path

DB_PATH = Path("../data/processed/db/lahman.db")
conn = sqlite3.connect(DB_PATH)
conn

In [3]:
query = """
SELECT 
    b.playerID,
    b.yearID,
    p.nameFirst,
    p.nameLast,

    -- basic counting stats
    SUM(b.G)      AS G,
    SUM(b.AB)     AS AB,
    SUM(b.R)      AS R,
    SUM(b.H)      AS H,
    SUM(b."2B")   AS doubles,
    SUM(b."3B")   AS triples,
    SUM(b.HR)     AS HR,
    SUM(b.RBI)    AS RBI,
    SUM(b.SB)     AS SB,
    SUM(b.CS)     AS CS,
    SUM(b.BB)     AS BB,
    SUM(b.IBB)    AS IBB,
    SUM(b.SO)     AS SO,
    SUM(b.HBP)    AS HBP,
    SUM(b.SH)     AS SH,
    SUM(b.SF)     AS SF,
    SUM(b.GIDP)   AS GIDP

FROM Batting b
JOIN People p USING (playerID)
WHERE b.yearID BETWEEN 2020 AND 2025
GROUP BY b.playerID, b.yearID
"""

lahman_hitters = pd.read_sql_query(query, conn)
lahman_hitters.head()

,playerID,yearID,nameFirst,nameLast,G,AB,R,H,doubles,triples,...,RBI,SB,CS,BB,IBB,SO,HBP,SH,SF,GIDP
0,abadfe01,2021,Fernando,Abad,16,0,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,abadfe01,2023,Fernando,Abad,6,0,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,abbotan01,2023,Andrew,Abbott,21,0,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,abbotan01,2024,Andrew,Abbott,25,0,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
4,abbotco01,2021,Cory,Abbott,8,3,0,1,0,0,...,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df = lahman_hitters.copy()

# keep only rows with at least 1 AB
df = df[df["AB"] > 0].copy()

# fill NaNs for safety
for col in ["HBP", "SF", "SH"]:
    if col in df.columns:
        df[col] = df[col].fillna(0)
    else:
        df[col] = 0

# 1) Plate Appearances
df["PA"] = df["AB"] + df["BB"] + df["HBP"] + df["SF"] + df["SH"]
df["PA"] = df["PA"].clip(lower=1)

# 2) AVG
df["AVG"] = df["H"] / df["AB"]

# 3) OBP
df["OBP"] = (df["H"] + df["BB"] + df["HBP"]) / (
    df["AB"] + df["BB"] + df["HBP"] + df["SF"]
)

# 4) Total bases using doubles/triples aliases
df["TB"] = (
    (df["H"] - df["doubles"] - df["triples"] - df["HR"])  # singles
    + 2 * df["doubles"]
    + 3 * df["triples"]
    + 4 * df["HR"]
)

# 5) SLG
df["SLG"] = df["TB"] / df["AB"]

# 6) OPS
df["OPS"] = df["OBP"] + df["SLG"]

# 7) Per-PA rates
df["HR_rate"] = df["HR"] / df["PA"]
df["BB_rate"] = df["BB"] / df["PA"]
df["K_rate"]  = df["SO"] / df["PA"]

df[
    [
        "nameFirst", "nameLast", "yearID",
        "AB", "H", "HR",
        "AVG", "OBP", "SLG", "OPS",
        "HR_rate", "BB_rate", "K_rate"
    ]
].head()

KeyError: '2B'

In [ ]:
FEATURES_DIR = Path("../data/processed/features/")
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

output_path = FEATURES_DIR / "lahman_hitting_2020_2025.csv"
df.to_csv(output_path, index=False)
output_path

PosixPath('../data/processed/features/lahman_hitting_2020_2025.csv')

In [ ]:
statcast_path = FEATURES_DIR / "statcast_hitting_2020_2025.csv"
statcast = pd.read_csv(statcast_path)

statcast.head()
statcast.columns

Index(['last_name, first_name', 'player_id', 'attempts', 'avg_hit_angle',
       'anglesweetspotpercent', 'max_hit_speed', 'avg_hit_speed', 'ev50',
       'fbld', 'gb', 'max_distance', 'avg_distance', 'avg_hr_distance',
       'ev95plus', 'ev95percent', 'barrels', 'brl_percent', 'brl_pa', 'year'],
      dtype='object')